# ADFT Input Examples

In this demo, we submit a calculation, check the status of the job and query the results after it is finished.    

We demonstrate different options for building input for ADFT.

In [1]:
from azure.quantum import Workspace
from azure.quantum.job import JobFailedWithResultsError

# insert connection string form Azure Portal Workspace Access Keys
connection_string = ""

workspace = Workspace.from_connection_string(connection_string)

In [2]:
# To submit Accelerated DFT jobs, we will be using the microsoft.dft target in the workspace.
print("Verifying access to Accelerated DFT target.")
target = workspace.get_targets("microsoft.dft")
print("Verification complete.")

Verifying access to Accelerated DFT target.
Verification complete.


# Section 1: QCSchema Input

ADFT supports the QCSchema input format, an open standard for compatibility across computational chemistry software.    
An example of this format is shown below. Note the 'driver' here is 'energy', which will perform a single point energy calculation.    
The driver may also be 'gradient', 'hessian', 'go' or 'bomd', see the documentation for more detail.     

In this format the atom labels and coordinates are separated into separate arrays. Note that QCSchema coordinates are in Bohr.        


In [27]:
qcschema_input = {
    "driver": "energy",
    "model": {
        "method": "m06-2x",
        "basis": "def2-svpd"
    },
    "schema_name": "qcschema_input",
    "schema_version": 1,
    "molecule": {
        "extras": {},
        "symbols": [
            "O",
            "C",
            "C",
            "C",
            "C",
            "C",
            "C",
            "H",
            "H",
            "H",
            "H",
            "H",
            "H"
        ],
        "geometry": [
            4.730542147965709,
            0.034826575331843086,
            0.07810088784463559,
            2.1361232242687977,
            0.017709001458524106,
            0.009088108672780787,
            0.7996954919209014,
            2.290483253979806,
            0.10106814673106823,
            -1.8298562750208616,
            2.2732950799384737,
            -0.04537958079912547,
            -3.1327572801516967,
            -0.00564083248182671,
            -0.28742004920350506,
            -1.790388872477789,
            -2.271959799458856,
            -0.38978844089184156,
            0.8394687277399734,
            -2.2656284043593296,
            -0.24392044354214196,
            5.279447115915874,
            -0.07938333158181043,
            1.8109098053069272,
            1.8583211818406624,
            4.051452964636673,
            0.2691141588512759,
            -2.8675310249318393,
            4.053900197762506,
            0.0241508699472927,
            -5.190440656400895,
            -0.014523603513912258,
            -0.4052054313284032,
            -2.796624853566738,
            -4.060585444078858,
            -0.5909607661605761,
            1.9285725820008635,
            -4.013248220398251,
            -0.3415529925897059
        ],
        "molecular_charge": 0, 
        "molecular_multiplicity": 1
    }
}

In [28]:
job_1 = target.submit(input_data=[qcschema_input],name='phenol_spe_method_1')
print("\nDFT job has been submitted.")
job_1.wait_until_completed()
print("\nDFT job completed.")


DFT job has been submitted.
.............
DFT job completed.


# Section 2: Submitting using an XYZ file and a partial Schema


ADFT calculations can also be submitted using an xyz file and a partial QCSchema input via use of the Azure-Quantum SDK.    
Note that the charge and mutliplicity cannot be changed from neutral singlet and it is suggested to use method 1 or 3. 

In [11]:
import glob

# Next, we create a dictionary variable to specify the parameters for the DFT calculation. 
params = {
    "driver": "energy",
    "model": { "method": "m06-2x", "basis": "def2-svpd" },
    "keywords": {
        "max_scf_steps": 100,
        "convergence_threshold": 1e-8,
    },
}

# specify the molecule:
xyz_file = glob.glob('molecules/phenol.xyz')

# We are now ready to submit the Job using the target.submit call. It takes three parameters-
# 1. The input molecule in xyz format.
# 2. The DFT parameters that we declared above.
# 3. A friendly name to help identify the job in the Azure Portal later.

job_2 = target.submit(
    input_params=params,
    #input_data=['molecules/phenol.xyz'],
    input_data=xyz_file,
    name='phenol_spe_method_2',
)

print("\nDFT job has been submitted.")
job_2.wait_until_completed() # this would wait until each ADFT calculation is completed
print("\nDFT job completed.")


DFT job has been submitted.
.............
DFT job completed.


# Section 3: Building a QCSchema Input Using Azure_quantum SDK

The main input of ADFT is the QCSchema format.    
It is therefore advantageous to use this format and offers the most flexibility in input.       
Additionally, if the charge and multiplicity are to be altered from a neutral singlet the QCSchema is the way to do this.

In [14]:
xyz_files = ['molecules/phenol.xyz']
params = {
    "driver": "energy",
    "model": { 
        "method": "m06-2x", 
        "basis": "def2-svpd" 
    }
}

# create input qcschema
qcschema_input = target.assemble_qcschema_from_files(xyz_files, params)

# view the fields using qcschema_input.keys()

# specifiy charge and multiplicity
#  (note the index '0' as we only have a single qcschema) 
qcschema_input[0]['molecule']['molecular_charge'] = 0
qcschema_input[0]['molecule']['molecular_multiplicity'] = 1 

In [15]:
job_3 = target.submit(input_data=qcschema_input,name='phenol_spe_method_3')
print("\nDFT job has been submitted.")
job_3.wait_until_completed()
print("\nDFT job completed.")


DFT job has been submitted.
.............
DFT job completed.


# Query Job Status and Retreive Results

We can retrieve information about a job through [Workspace.get_job](https://learn.microsoft.com/en-us/azure/quantum/optimization-job-management), and also query the results by filtering the job name with [Workspace.list_jobs](https://learn.microsoft.com/en-us/azure/quantum/optimization-job-management).

In [29]:
results_1 = job_1.get_results()
results_2 = job_2.get_results()
results_3 = job_3.get_results()

# QCSchema Output
qcschema_1 = results_1["results"][0]
qcschema_2 = results_2["results"][0]
qcschema_3 = results_3["results"][0]

In [30]:
# The energy can be accessed:
print("Method 1, Total Energy (Hartree): {:.8f}".format(qcschema_1['properties']['return_energy']))
# The energy can be accessed:
print("Method 2, Total Energy (Hartree): {:.8f}".format(qcschema_2['properties']['return_energy']))
# The energy can be accessed:
print("Method 3, Total Energy (Hartree): {:.8f}".format(qcschema_3['properties']['return_energy']))

Method 1, Total Energy (Hartree): -307.12139308
Method 2, Total Energy (Hartree): -307.12139308
Method 3, Total Energy (Hartree): -307.12139308
